In [1]:
import pandas as pd

In [2]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df = df.iloc[:300]

In [4]:
model_name='multi-qa-mpnet-base-dot-v1'
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
answer_llm = df.iloc[0].answer_llm
answer_vect = embedding_model.encode(answer_llm)
answer_vect[0]

-0.42244655

In [7]:
df.columns

Index(['answer_llm', 'answer_orig', 'document', 'question', 'course'], dtype='object')

In [8]:
import numpy as np
evaluations = []
for index, row in df.iterrows():
    answer_llm_vect = embedding_model.encode(row['answer_llm'])
    answer_orig_vect = embedding_model.encode(row['answer_orig'])
    dot = np.dot(answer_llm_vect, answer_orig_vect)
    evaluations.append(dot)

In [9]:
evaluations[0]

17.515987

In [10]:
np.percentile(evaluations, 75)

31.67430877685547

In [11]:
def normalize(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [12]:
evaluations = []
for index, row in df.iterrows():
    answer_llm_vect = embedding_model.encode(row['answer_llm'])
    answer_orig_vect = embedding_model.encode(row['answer_orig'])
    dot = np.dot(normalize(answer_llm_vect), normalize(answer_orig_vect))
    evaluations.append(dot)

In [13]:
np.percentile(evaluations, 75)

0.8362348973751068

In [14]:
%pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
r = df.iloc[10]
r

answer_llm     Yes, all sessions are recorded, so if you miss...
answer_orig    Everything is recorded, so you won’t miss anyt...
document                                                5170565b
question                    Are sessions recorded if I miss one?
course                                 machine-learning-zoomcamp
Name: 10, dtype: object

In [19]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [20]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [21]:
np.average([scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']])

0.35490034990035496

In [23]:
scores_rouge_2 = []
for index, row in df.iterrows():
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    scores_rouge_2.append(scores['rouge-2']['f'])

In [25]:
np.average(scores_rouge_2)

0.20696501983423318